In [37]:
pip install torch torch_geometric


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [38]:
pip install torch-geometric torch-scatter torch-sparse


  Using cached torch_scatter-2.1.2.tar.gz (108 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached torch_sparse-0.6.18.tar.gz (209 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py install for torch-scatter: started
  Running setup.py install for torch-scatter: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  DEPRECATION: torch-scatter is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559
  error: subprocess-exited-with-error
  
  × Running setup.py install for torch-scatter did not run successfully.
  │ exit code: 1
  ╰─> [35 lines of output]
      running install
      c:\Users\shawn\AppData\Local\Programs\Python\Python311\Lib\site-packages\setuptools\command\install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
        warnings.warn(
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-311
      creating build\lib.win-amd64-cpython-311\torch_scatter
      copying torch_scatter\placeholder.py -> 

In [2]:
import torch
import torch.nn as nn
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, global_mean_pool
import numpy as np
import pickle
import pandas as pd
import torch.nn.functional as F

In [6]:
# Load node features (gene expression)
node_features = np.load('node_features.npy', allow_pickle=True)
x = torch.tensor(node_features, dtype=torch.float)

# Load the JSD adjacency matrix
adj_matrix = np.load('adj_matrix.npy')

# Load the labels
labels = np.load('labels.npy')

# Convert labels to a tensor
y = torch.tensor(labels, dtype=torch.long)

print("Adjacency shape:", adj_matrix.shape)
print("Non-zero edges:", np.count_nonzero(adj_matrix))


Adjacency shape: (995, 995)
Non-zero edges: 989030


In [18]:
# Convert adjacency matrix to edge indices
def adj_to_edge_index(adj):
    edge_index = []
    n = adj.shape[0]
    for i in range(n):
        for j in range(n):
            if adj[i, j] > 0:
                edge_index.append([i, j])
    return torch.tensor(edge_index).t().contiguous()

threshold = 0.5  # Try adjusting this
sparse_adj = np.where(adj_matrix > threshold, adj_matrix, 0)
edge_index = adj_to_edge_index(sparse_adj)

# Create PyTorch Geometric data object
data = Data(x=x, edge_index=edge_index, y=y)

print(f"Data x shape: {data.x.shape}")
print(f"Data y shape: {data.y.shape}")

print("Edge index shape:", edge_index.shape)  # Should be [2, num_edges]
print("First 5 edges:", edge_index[:, :5])
num_classes = torch.unique(data.y).shape[0]
print("Number of classes:", num_classes)


Data x shape: torch.Size([995, 25])
Data y shape: torch.Size([995])
Edge index shape: torch.Size([2, 172264])
First 5 edges: tensor([[ 0,  0,  0,  0,  0],
        [16, 17, 26, 29, 38]])
Number of classes: 1


In [19]:
class BayesianDeeperGCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, dropout_rate=0.3):
        super(BayesianDeeperGCN, self).__init__()
        
        # GCN layers
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, out_channels)  # Output 1 for regression (risk score)
        
        self.dropout_rate = dropout_rate
    
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        # Pass through first GCN layer with ReLU activation
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=self.dropout_rate, training=self.training)
        
        # Pass through second GCN layer with ReLU activation
        x = F.relu(self.conv2(x, edge_index))
        x = F.dropout(x, p=self.dropout_rate, training=self.training)
        
        # Pass through third GCN layer for output (no activation for regression)
        x = self.conv3(x, edge_index)
        
        return x  # Output the raw continuous values


Edge index shape: torch.Size([2, 989030])
First 5 edges: tensor([[0, 0, 0, 0, 0],
        [1, 2, 3, 4, 5]])


This indiactes that the non-zero edges are almost fully connected graph which accumulates to 989,030 possible edges which is a concern. 

Which we then created a threshold, and sparse_adj to remove weak edges which significantly reduced the number of connected graph to 172,264 total directed edges. 

Edge index shape: torch.Size([2, 172264])
First 5 edges: tensor([[ 0,  0,  0,  0,  0],
                       [16, 17, 26, 29, 38]])

Which states that the first 5 edges node 0 connects to 16,17,26,29 and 38

edge_index = adj_to_edge_index(sparse_adj)
Which makes the GCn have new sparser edge connections, which helps with
- Reducing overfitting
- Speed up training
- Possibly improve generalization


We are using classificaiton, which then we will be moving into regression to predict risk scores and see if donors have tells whether or not they are leading towards cancer.

Moving from Classification to Regression
- Classification involves predicting discrete labels. In the case of the project, doing a classification task, we would output a label that tells which class the sample belongs to.
- Regression, is more suited for predicting continuous values, such as risk scores (a score between 0 and 1 representing the likelihood of cancer, or even a continuous prediction of gene expression levels).
By switching to regression, the model will predict ocntinuous values instead of discrete class labels. This will allow the assessment of how far a particular gene expression profile is from a "normal" or "healthy" state.

===================

Learning rate = 0.001
Epoch 0: Loss = 0.050308242440223694
Epoch 10: Loss = 0.004108337685465813
Epoch 20: Loss = 0.0009809117764234543
Epoch 30: Loss = 0.0004824417410418391
Epoch 40: Loss = 0.00037993560545146465
Epoch 50: Loss = 0.0002862958353944123
Epoch 60: Loss = 0.00035408855183050036
Epoch 70: Loss = 0.00020732838311232626
Epoch 80: Loss = 0.0008582030422985554
Epoch 90: Loss = 0.0003825158637482673

Learning rate = 0.01
Epoch 0: Loss = 0.5243377685546875
Epoch 10: Loss = 0.028047975152730942
Epoch 20: Loss = 0.004144496284425259
Epoch 30: Loss = 0.00024922509328462183
Epoch 40: Loss = 0.0003573083959054202
Epoch 50: Loss = 0.00011655231355689466
Epoch 60: Loss = 0.000460177194327116
Epoch 70: Loss = 0.00036572854151017964
Epoch 80: Loss = 0.00015529152005910873
Epoch 90: Loss = 0.0003323396376799792


The difference in the loss progression between the two learning rate (0.001 and 0.01) can be attributed to the size of the learning rate and how it impacts the optimization process.

Smaller Learning Rate (0.001):
- The model starts with a relatively smaller loss at epoch 0 (0.0503), and the loss decrease steadily and more smoothly throughout the epochs.
- This suggests the model is gradually adjusting its parameters in smaller increments, leading to steady progress toward a lower loss.
Larger Learning Rate (0.01):
- The model starts with a much higher loss at epoch 0 (0.5243) which is significantly larger than the loss when the learning rate is smaller.
- However, the loss decreases rapidly in the intial epochs, espcially at epoch 10 (0.0280) indicating that the model is making larger updates and potentially converging faster early on.
- However, from epoch 40 onwards, the loss fluctautes a bit and doesn't decresae as smoothly as with smaller learning rate. The loss starts to rise again in some epochs suggesting that the larger learning rate may be causing instability in the optimization process

Learning Rate and Convergence:
- If the learning rate is too large, the model can take steps that are too large, leading to poor generalization and unstable training. A smaller learning rate, on the other hand, allows the model to make more controlled progress, leading to smoother convergence.

Epoch 0: Loss = 0.19076433777809143
Epoch 10: Loss = 0.08978673070669174
Epoch 20: Loss = 0.0577743798494339
Epoch 30: Loss = 0.0066403551027178764
Epoch 40: Loss = 0.001223327242769301
Epoch 50: Loss = 0.0008435531053692102
Epoch 60: Loss = 0.00032519144588150084
Epoch 70: Loss = 0.0001917402696562931
Epoch 80: Loss = 0.0007538400823250413
Epoch 90: Loss = 0.0004983903490938246

Observations:
Initial Loss Decrease:
- The loss starts at 0.1907 and drops steadily to 0.0005 by epoch 90. 
Gradual Decrease:
- The loss decreases gradually and steadily where it drops from 0.1907 to 0.0066 by epoch 30 indicating that the model is learning at a reasonable direction in parameter space.
Stability:
- The loss doesn't fluctuate significantly. Suggesting taht learning rate of 0.0001 is small enough to allow the optimizer to make gradual adjustments, avoiding the potential for overshooting or instability seen with larger learning rates.


In [22]:
# Training loop for regression
def train(model, data, optimizer, criterion, epochs=100):
    model.train()  # Set model to training mode
    for epoch in range(epochs):
        optimizer.zero_grad()
        
        # Forward pass
        out = model(data)  # The model now outputs continuous values
        
        # Compute loss (using MSE for regression)
        loss = criterion(out.squeeze(), data.y.float())  # Make sure data.y is float
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        if epoch % 10 == 0:
            print(f"Epoch {epoch}: Loss = {loss.item()}")

# Instantiate the model
in_channels = x.shape[1]  # Number of features (gene expression values)
hidden_channels = 64  # You can change this to a higher value if needed
out_channels = 1  # We are predicting a single risk score for regression

model = BayesianDeeperGCN(in_channels, hidden_channels, out_channels)

# Loss function and optimizer for regression
criterion = nn.MSELoss()  # Mean Squared Error for regression
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

# Train the model
train(model, data, optimizer, criterion, epochs=100)

Epoch 0: Loss = 0.19076433777809143
Epoch 10: Loss = 0.08978673070669174
Epoch 20: Loss = 0.0577743798494339
Epoch 30: Loss = 0.0066403551027178764
Epoch 40: Loss = 0.001223327242769301
Epoch 50: Loss = 0.0008435531053692102
Epoch 60: Loss = 0.00032519144588150084
Epoch 70: Loss = 0.0001917402696562931
Epoch 80: Loss = 0.0007538400823250413
Epoch 90: Loss = 0.0004983903490938246
